Copyright (c) 2023 Habana Labs, Ltd. an Intel Company.

#### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Using LocalGPT on the Intel&reg; Gaudi&reg; 2 AI accelerator with the Llama2 70B model
This tutorial will show how to use the [LocalGPT](https://github.com/PromtEngineer/localGPT) open source initiative on the Intel Gaudi 2 AI accelerator.  LocalGPT allows you to load your own documents and run an interactive chat session with this material.  This allows you to query and summarize your content by loading any .pdf or .txt documents into the `SOURCE DOCUMENTS` folder, running the ingest.py script to tokenize your content and then the run_localGPT.py script to start the interaction.  

In this example, we're using the **meta-llama/Llama-2-70b-chat-hf** model as the refrence model that will manage the inference on Gaudi 2.  DeepSpeed inference is used based on the size of the model.

To optimize this instantiation of LocalGPT, we have created new content on top of the existing Hugging Face based "text-generation" inference task and pipelines, including:

1. Using the Hugging Face Optimum Habana Library with the Llama2-70B model, which is optimized on Gaudi2. 
2. Using Langchain to import the source docuement with a custom embedding model, using a `GaudiHuggingFaceEmbeddings` class based on HuggingFaceEmbeddings.
3. We are using a custom pipeline class, `GaudiTextGenerationPipeline` that optimizes text-generation tasks for padding and indexing for static shapes, to improve performance.


##### Install DeepSpeed to run inference on the full Llama2 70B model

In [1]:
!pip install git+https://github.com/HabanaAI/DeepSpeed.git@1.14.0

  Cloning https://github.com/HabanaAI/DeepSpeed.git (to revision 1.13.0) to /tmp/pip-req-build-ptlbmyw4
  Running command git clone --filter=blob:none --quiet https://github.com/HabanaAI/DeepSpeed.git /tmp/pip-req-build-ptlbmyw4
  Running command git checkout -b 1.13.0 --track origin/1.13.0
  Switched to a new branch '1.13.0'
  Branch '1.13.0' set up to track remote branch '1.13.0' from 'origin'.
  Resolved https://github.com/HabanaAI/DeepSpeed.git to commit 6522014efac08fdcbc37ea4a9d85552ce9cb7b50
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.4 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.10.3+hpu.synapse.v1.13.0-py3-none-any.whl size=941404 sha256=d5d3cac4bb0f61e408f53b3c0f1e228bd435a27efa4fe8139075622cff6b77ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-insmzxsu/wheels/80/ad/7c/2b51ee0f18d4027d2c8ef64ef28ab3a3943c46d606ab4417f4
Successfully built deepspeed


##### Go to the LocalGPT folder and set environment varialbles

In [2]:
%cd /root/Gaudi-tutorials/PyTorch/localGPT_inference
!export DEBIAN_FRONTEND="noninteractive"
!export TZ=Etc/UTC

/root/Gaudi-tutorials/PyTorch/localGPT_inference


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


##### Install the requirements for LocalGPT 

In [3]:
!apt-get update
!apt-get install -y tzdata bash-completion python3-pip openssh-server      vim git iputils-ping net-tools protobuf-compiler curl bc gawk tmux     && rm -rf /var/lib/apt/lists/*
!pip install -q --upgrade pip
!pip install -q -r requirements.txt

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.0 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,512 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,036 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB] 
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,282 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1,792 kB]
Get:13 http://archive.ubuntu.com/ubunt

##### Install the Optimum Habana Library from Hugging Face

In [4]:
!pip install -q optimum-habana==1.10.0

### Load your Local Content
Copy all of your files into the `SOURCE_DOCUMENTS` directory

The current default file types are .txt, .pdf, .csv, and .xlsx, if you want to use any other file type, you will need to convert it to one of the default file types.

Run the following command to ingest all the data. The ingest.py uses LangChain tools to parse the document and create embeddings locally using the GaudiHuggingFaceEmbeddings class. It then stores the result in a local vector database (DB) using Chroma vector store. 

If you want to start from an empty database, delete the DB folder and run the ingest script again. 

In [5]:
!python ingest.py --device_type hpu

2023-12-09 00:42:28,492 - INFO - ingest.py:124 - Loading documents from /root/Gaudi-tutorials/PyTorch/localGPT_inference/SOURCE_DOCUMENTS
2023-12-09 00:42:28,508 - INFO - ingest.py:37 - Loading document batch
2023-12-09 00:42:30,371 - INFO - ingest.py:133 - Loaded 1 documents from /root/Gaudi-tutorials/PyTorch/localGPT_inference/SOURCE_DOCUMENTS
2023-12-09 00:42:30,371 - INFO - ingest.py:134 - Split into 72 chunks of text
/usr/local/lib/python3.10/dist-packages/torch/distributed/distributed_c10d.py:252: UserWarning: Device capability of hccl unspecified, assuming `cpu` and `cuda`. Please specify it via the `devices` argument of `register_backend`.
  warnings.warn(
Loading Habana modules from /usr/local/lib/python3.10/dist-packages/habana_frameworks/torch/lib
2023-12-09 00:42:32,089 - INFO - SentenceTransformer.py:66 - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-12-09 00:42:36,740 - INFO - SentenceTransformer.py:66 - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
=

### How to access and Use the Llama 2 model
Use of the pretrained model is subject to compliance with third party licenses, including the “Llama 2 Community License Agreement” (LLAMAV2). For guidance on the intended use of the LLAMA2 model, what will be considered misuse and out-of-scope uses, who are the intended users and additional terms please review and read the instructions in this link https://ai.meta.com/llama/license/. Users bear sole liability and responsibility to follow and comply with any third party licenses, and Habana Labs disclaims and will bear no liability with respect to users’ use or compliance with third party licenses.

To be able to run gated models like this Llama-2-70b-chat-hf, you need the following:

* Have a HuggingFace account
* Agree to the terms of use of the model in its model card on the HF Hub
* Set a read token
* Login to your account using the HF CLI: run huggingface-cli login before launching your script

In [6]:
#!huggingface-cli login --token <your token here>

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Running the LocalGPT model with Llama2 70B Chat 

### Set the model Usage

To change the model, you can modify the "LLM_ID = <add model here>" in the `constants.py` file. For this example, the default is `meta-llama/Llama-2-70b-chat-hf`.  

Since this is interactive, it's a better experince to launch this from a terminal window.  This run_localGPT.py script uses a local LLM (Llama 2 in this case) to understand questions and create answers. The context for the answers is extracted from the local vector store using a similarity search to locate the right piece of context from the documentation.  This is the run command to use:

`PT_HPU_LAZY_ACC_PAR_MODE=1 PT_HPU_ENABLE_LAZY_COLLECTIVES=true python gaudi_spawn.py --use_deepspeed --world_size 8 run_localGPT.py --device_type hpu --temperature 0.7 --top_p 0.95`

Running the full 70B model takes up ~128GB of disk space, so if your system is storage constrained, it may be best to run the Llama 2 7B or 13B chat models.  Change the LLM_ID variable in the `constants.py` file (example: `LLM_ID = "meta-llama/Llama-2-7b-chat-hf"`) and use the command below.
`python run_localGPT.py --device_type hpu --temperature 0.7 --top_p 0.95`

Note: The inference is running sampling mode, so the user can optinally modify the temperature and top_p settings.  The current settings are temperature=0.7, top_p=0.95.  Type "exit" at the prompt to stop the execution.


In [ ]:
#Run this command in a terminal window to start the interactive chat: `PT_HPU_LAZY_ACC_PAR_MODE=1 PT_HPU_ENABLE_LAZY_COLLECTIVES=true python gaudi_spawn.py --use_deepspeed --world_size 8 run_localGPT.py --device_type hpu --temperature 0.7 --top_p 0.95`, the example below is showing the initial output:   

In [ ]:
!PT_HPU_LAZY_ACC_PAR_MODE=1 PT_HPU_ENABLE_LAZY_COLLECTIVES=true python gaudi_spawn.py --use_deepspeed --world_size 8 run_localGPT.py --device_type hpu --temperature 0.2 --top_p 0.95

DistributedRunner run(): command = deepspeed --num_nodes 1 --num_gpus 8 --no_local_rank run_localGPT.py --device_type hpu --temperature 0.2 --top_p 0.95
/usr/local/lib/python3.10/dist-packages/torch/distributed/distributed_c10d.py:252: UserWarning: Device capability of hccl unspecified, assuming `cpu` and `cuda`. Please specify it via the `devices` argument of `register_backend`.
  warnings.warn(
[2023-12-09 01:30:35,163] [INFO] [real_accelerator.py:175:get_accelerator] Setting ds_accelerator to hpu (auto detect)
[2023-12-09 01:30:36,512] [WARNING] [runner.py:206:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2023-12-09 01:30:36,575] [INFO] [runner.py:585:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMCwgMSwgMiwgMywgNCwgNSwgNiwgN119 --master_addr=127.0.0.1 --master_port=29500 --no_local_rank --enable_each_rank_log=None run_localGPT.py --device_type hpu --temperature 0.2 --top_p 0.95
/usr/lo

In [ ]:
exit()